In [1]:
import pandas as pd

import fastparquet

import gc

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
from pipe_utils import full_path, logme

In [5]:
pd.set_option('display.max_columns', None)

# Loading Partition 0

<i>Загрузим и проанализуируем для начала один из parquet-файлов

In [27]:
df = pd.read_parquet(full_path('data/train_data/train_data_0.pq'))
df

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,5,0,2,8,6,16,5,4,8,1,1,1,1,1,15,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,3,0,2,4,6,16,5,4,8,0,1,1,1,1,16,2,17,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974719,249998,4,1,9,14,8,10,11,8,4,2,0,2,4,6,16,5,4,8,1,1,1,1,1,11,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,1
1974720,249998,5,7,9,4,8,1,11,19,1,4,0,2,7,6,16,5,4,8,1,1,1,1,1,7,2,17,0,1,1,0,0,0,0,0,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,1,1
1974721,249999,1,9,0,10,8,10,11,16,2,3,0,2,7,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,1
1974722,249999,2,9,16,10,13,10,4,12,2,3,0,2,7,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,0


Merge Target:

In [8]:
target = pd.read_csv(full_path('data/train_target.csv'))
df2 = df.join(target.flag, how='left')
df2.rename(columns={'flag': 'target'}, inplace=True)
df2

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,target
0,0,1,18,9,2,3,16,10,11,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,0
1,0,2,18,9,14,14,12,12,0,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,3,4,1,0,0,0
2,0,3,18,9,4,8,1,11,11,0,5,0,2,8,6,16,5,4,8,1,1,1,1,1,15,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,2,3,1,1,1,0
3,0,4,4,1,9,12,16,7,12,2,3,0,2,4,6,16,5,4,8,0,1,1,1,1,16,2,17,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,1,1,0,0,0
4,0,5,5,12,15,2,11,12,10,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974719,249998,4,1,9,14,8,10,11,8,4,2,0,2,4,6,16,5,4,8,1,1,1,1,1,11,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,1,0
1974720,249998,5,7,9,4,8,1,11,19,1,4,0,2,7,6,16,5,4,8,1,1,1,1,1,7,2,17,0,1,1,0,0,0,0,0,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,1,1,1
1974721,249999,1,9,0,10,8,10,11,16,2,3,0,2,7,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,1,0
1974722,249999,2,9,16,10,13,10,4,12,2,3,0,2,7,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,0,1


## Data understading and feature selection

<i>Попробуем определить значимость фичей, используя оценку Фишера

<i>P.S. В итоге, после нескольких циклов моделирования я понял, что максимальную точность можно получить, если использовать все фичи

### Fisher's score

In [8]:
from skfeature.function.similarity_based import fisher_score

In [11]:
samp = df2.sample(10000)

In [12]:
rank = fisher_score.fisher_score(samp.drop(columns='target').to_numpy(), samp['target'].to_numpy())
feat_imp = pd.Series(rank, samp.drop(columns='target').columns)

In [17]:
feat_imp.sort_values(ascending=True).head(30)

enc_paym_24                       0
is_zero_loans530                  1
is_zero_over2limit                2
enc_paym_17                       3
enc_loans_account_holder_type     4
enc_paym_23                       5
pre_loans_outstanding             6
pre_since_confirmed               7
pre_loans_next_pay_summ           8
pre_fterm                         9
id                               10
pre_since_opened                 11
rn                               12
pre_pterm                        13
pre_till_fclose                  14
pre_loans_credit_limit           15
pre_loans_credit_cost_rate       16
pre_loans5                       17
pre_loans530                     18
pre_loans_total_overdue          19
pre_util                         20
is_zero_loans3060                21
pre_till_pclose                  22
is_zero_loans6090                23
pre_loans3060                    24
pre_over2limit                   25
enc_paym_7                       26
enc_paym_8                  

### Cramer's correlation

<i>Вычислив матрицу корреляций Крамера, посмотрим, нет ли линейно зависимых фичей

In [147]:
import scipy.stats as ss
import numpy as np

def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.to_numpy().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    rcorr += 0.000001 if rcorr == 1.0 else 0
    kcorr += 0.000001 if kcorr == 1.0 else 0
    return np.sqrt(phi2corr / min( kcorr-1, rcorr-1 ))

In [149]:
from itertools import product

for colX, colY in product(df.drop(columns=['id', 'rn']).columns, repeat=2):
    confusion_matrix = pd.crosstab(df[colX], df[colY])
    cramer_matrix.loc[colX, colY] = cramers_corrected_stat(confusion_matrix)


In [152]:
corr_cols = [k for k, v in cramer_matrix[(cramer_matrix <= 0.999) & (cramer_matrix > 0.7)].notna().any().items() if v]

In [153]:
cramer_matrix[cramer_matrix > 0.7].loc[corr_cols, corr_cols]

,pre_pterm,pre_fterm,pre_loans_credit_limit,pre_util,pre_over2limit,pre_maxover2limit,enc_loans_credit_status,pre_till_pclose,pre_till_fclose,pre_loans_outstanding,pre_loans_max_overdue_sum,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag
pre_pterm,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.942130,NaN
pre_fterm,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.979666
pre_loans_credit_limit,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.704909,NaN,NaN,NaN
pre_util,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.970562,0.940954,NaN,NaN,NaN
pre_over2limit,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.971712,NaN,NaN,NaN
pre_maxover2limit,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.955972,0.785576,NaN,NaN
enc_loans_credit_status,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.746591,NaN,NaN,NaN,NaN
pre_till_pclose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.870175,NaN
pre_till_fclose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,0.704827,NaN,NaN,NaN,0.889790
pre_loans_outstanding,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.780517,NaN,NaN,NaN,NaN


<i>Есть корреляции между некоторомы фичами, но, как я уже писал, я остановился на всех фичах, так как похоже, что корреляции не имеют влияние на выбранные в итоге типы моделей


### enc_paym_N

<i>рассмотрим значения enc_paym_*

In [9]:
enc_paym =[x for x in df2.columns if x.startswith('enc_paym')]

In [ ]:
for col in enc_paym:
    temp = df2[col].value_counts()
    print(temp, '\n')

In [ ]:
temp = pd.DataFrame(index=enc_paym, columns=range(5))
for col in enc_paym:
    temp.loc[col] = df2[col].value_counts()
    
temp

<i>Видно, что значения для месяцев 11, 20, 24 сдвинуты на единицу. Для генераии фич из этих колонок нужно, чтобы они были закодированы одинаково

In [ ]:
for col in ['enc_paym_11', 'enc_paym_20', 'enc_paym_24']:
    df2[col] = df2[col].apply(lambda x: x - 1)

In [ ]:
df2[enc_paym].sample(n=5)

In [ ]:
temp = pd.DataFrame(index=enc_paym, columns=range(5))
for col in enc_paym:
    temp.loc[col] = df2[col].value_counts()

In [ ]:
plt.figure(figsize=(12,6))
plt.tick_params(axis='x', labelrotation=90)
sns.scatterplot(temp)
;

<i>Теперь значения корректны, попробуем определить, что значат коды 0, 1, 2, 3

In [ ]:
col_flags = ['is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90']

In [ ]:
df2[(df2[col_flags] == 0).any(axis=1)]

In [ ]:
df2[(df2[enc_paym] == 1).any(axis=1) & (df2[col_flags] == 0).any(axis=1)].shape

In [ ]:
df2[(df2[enc_paym] == 2).any(axis=1) & (df2[col_flags] == 0).any(axis=1)].shape

Делаем следующий вывод:
* 0 - всё нормально
* 3 - кредит закрыт или ещё не открыт
* 1 - просрочка
* 2 - просрочка

### Max loan months

<i>Ведём фичу max loan months, показывающую максимальные период просрочки платежа (коды 1, 2) в месяцах

In [10]:
def max_loan_months(s: pd.Series) -> int:
    chunks = [[]]
    for val in s.values:
        if val not in (1, 2):
            chunks.append([])
        else:
            chunks[-1].append(val)
    return max(map(len, chunks))

In [ ]:
df2['max_loan_months'] = df2[enc_paym].apply(max_loan_months, axis=1)
df2

In [ ]:
df2.max_loan_months.value_counts()

### Frac loan months

<i>Введём фичу Frac loan months, показывающую долю месяцев с просрочками к кол-ву всех платёжных месяцев

In [11]:
def frac_loan_months(df: pd.DataFrame) -> pd.DataFrame:
    total_pay_months = (df != 3).sum(axis=1)
    total_pay_months = total_pay_months.apply(lambda x: 24 if x == 0 else x)
    return ((df == 1).sum(axis=1) + (df == 2).sum(axis=1)) / total_pay_months

In [ ]:
df2['frac_loan_months'] = frac_loan_months(df2[enc_paym])
df2.head()

### Credit History length

<i>Введём для каждого кредита Credit history length, покзывающую кол-во месяцев, которое уже выплачивается кредит

In [12]:
def credit_history_length(s: pd.Series) -> int:
    for col in reversed(enc_paym):
        if s[col] != 3: 
            return int(col.lstrip('enc_paym_'))
    return 0

credit_history_length(df2.loc[4, enc_paym])

In [ ]:
df2.loc[4, enc_paym]

In [ ]:
df2['credit_history_length'] = df2[enc_paym].apply(credit_history_length, axis=1)
df2.head()

In [ ]:
df = df2.copy()

### OHE и агрегация

In [ ]:
for col in df.columns:
    print(f"{col}\n{df[col].value_counts()}\n")

In [ ]:
flags = [
    'is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90', 
    'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit', 'pclose_flag', 'fclose_flag', 
]
num = ['rn', 'credit_history_length', 'max_loan_months', 'frac_loan_months']
cat = [x for x in df.columns if x not in ['id', 'rn'] + flags + num + ['pre_loans6090', 'pre_over2limit', 'enc_loans_account_cur']]

cols = ['id'] + cat + num + flags
enc_paym = [x for x in df.columns if x.startswith('enc_paym')]

In [ ]:
ohe = OneHotEncoder(sparse_output=False, dtype='int8')

In [ ]:
df2 = df[cols].drop(columns=cat).join(
    pd.DataFrame(
        ohe.fit_transform(df[cat]),
        index=df.index,
        columns=ohe.get_feature_names_out()
    )
)
df2

In [ ]:
enc = set(df2.columns) - set(num + flags + ['rn', 'id'])

<i>Агрегируем все закодированные фичи суммированием для каждого id

<i>Флаги также агрегируем суммированием

<i>Для числовых фичей возьмём максимальные значения

In [ ]:
func_dict = dict.fromkeys(enc, 'sum')
func_dict.update(dict.fromkeys(flags, 'sum'))
func_dict.update(dict.fromkeys(num, 'max'))

In [ ]:
df_agg = df2.groupby('id').agg(func_dict)
df_agg

------

## has loans

<i>Посмотрим распределение классов таргета среди допустивших любую просрочку

In [ ]:
cols_flag = [col for col in df_agg.columns if col.startswith('is_zero_loans')]
cols_flag

In [ ]:
df2

In [ ]:
temp = df2.target.value_counts()
print('Отношение во всей выборке:')
temp[0] / temp[1]

In [ ]:
temp = df2[(df2[cols_flag] == 0).any(axis=1)].target.value_counts()
print('Отношение среди допустивших просрочку:')
temp[1] / temp[0]

<i>можно ввести бинарную фичу has_loans, показывающую наличие хотя бы одной просрочки по кредиту

In [ ]:
df_agg.loc[:, 'has_loans'] = (df_agg[cols_flag] == 0).any(axis=1).astype('int8')
df_agg.head()

In [ ]:
df_agg.has_loans.value_counts()

## Relative one-hot encoded features

In [ ]:
coded = df_agg.drop(columns=['rn', 'credit_history_length', 'max_loan_months', 'frac_loan_months', 'has_loans']).columns

In [ ]:
df_coded = df_agg[coded].div(df_agg.rn, axis=0)

In [ ]:
df_agg[coded] = df_coded
df_agg

In [ ]:
del df_coded
gc.collect()

# All partitions

<i>Сделаем функцию для агрегации всего датасета

In [14]:
VERBOSE = True

class Aggregator:
    def __init__(self, df: pd.DataFrame, verbose=False):
        self.df = df
        self.df_agg = pd.DataFrame(index=df.id.unique())
        self.verbose = verbose
    
        self.flags = [
            'is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90', 
            'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit', 'pclose_flag', 'fclose_flag', 
        ]
        self.enc_paym = [x for x in self.df.columns if x.startswith('enc_paym')]
        self.cat = [x for x in self.df.columns if x not in ['id', 'rn'] + self.flags + ['pre_loans6090', 'pre_over2limit', 'enc_loans_account_cur']]
        self.num = ['rn', 'credit_history_length', 'num_loan_months', 'frac_loan_months']
    
    
    def __str__(self):
        return f"{self.df_agg.__class__.__name__}{self.df_agg.shape}"
    
   
    @staticmethod
    def max_loan_months(s: pd.Series) -> int:
        chunks = [[]]
        for val in s.values:
            if val not in (1, 2):
                chunks.append([])
            else:
                chunks[-1].append(val)
        return max(map(len, chunks))
    
    
    @staticmethod
    def frac_loan_months(df: pd.DataFrame) -> pd.DataFrame:
        total_pay_months = (df != 3).sum(axis=1)
        total_pay_months = total_pay_months.apply(lambda x: 24 if x == 0 else x)
        return ((df == 1).sum(axis=1) + (df == 2).sum(axis=1)) / total_pay_months
    
    
    @staticmethod    
    def credit_history_length(s: pd.Series, enc_paym: list) -> int:
        for col in reversed(enc_paym):
            if s[col] != 3: 
                return int(col.lstrip('enc_paym_'))
        return 0
    
    
    ## Flags
    @logme('Aggregate flags: ', verbose=VERBOSE)
    def __agg_flags(self):
        self.df_agg.loc[:,self.flags] = self.df[['id'] + self.flags].groupby('id').agg('sum')
        if self.verbose: print('▮', end='')
        
        
    ## Categorial encode
    @logme('Categorial encoding: ', verbose=VERBOSE)
    def __agg_cat(self):
        for col in self.cat:

            res = pd.DataFrame({'id': self.df.id, col: self.df[col]}, index=self.df.index)

            ohe = OneHotEncoder(sparse_output=False, dtype='int8')

            res = res.join(
                pd.DataFrame(
                    ohe.fit_transform(res[[col]]),
                    index=res.index,
                    columns=ohe.get_feature_names_out()
                )
            )

            res = res.drop(columns=col).groupby('id').agg('sum')   
            self.df_agg = self.df_agg.join(res, on=self.df_agg.index, how='left')

            if self.verbose: print('▮', end='')

    def __chunked_apply(self, chunk_size: int, feature_title: str, *args, **kwargs):
        """make new feature <feature_title>, applying pd.DataFrame.apply() with *args/**kwargs chunk by chunk"""
        n = 0
        while n < self.df.shape[0]:
            n_init = n
            n = min(n + chunk_size, self.df.shape[0])
            chunk = self.df[self.enc_paym].iloc[n_init:n]\
                    .apply(*args, **kwargs).astype('int')
            self.df.loc[n_init:n, feature_title] = chunk
            if self.verbose: print(n // chunk_size, end='')
    
    
    ## F.eng from enc_paym_N
    @logme('Features from enc_paym_N: ', verbose=VERBOSE)
    def __new_from_enc_paym(self):
        for col in ['enc_paym_11', 'enc_paym_20', 'enc_paym_24']:
            self.df[col] = self.df[col].apply(lambda x: x - 1)
        if self.verbose: print('▮', end='')

        self.df['num_loan_months'] = self.df[self.enc_paym].apply(__class__.max_loan_months, axis=1)
        if self.verbose: print('▮', end='')
        
        self.df['frac_loan_months'] = __class__.frac_loan_months(self.df[self.enc_paym])
        if self.verbose: print('▮', end='')

        self.__chunked_apply(3_000_000, 'credit_history_length',
                             __class__.credit_history_length,
                             args=(self.enc_paym, ),
                             axis=1)
        if self.verbose: print('▮', end='')

    
    ## Aggregate numerical -> max
    @logme('Aggregate numerical ', verbose=VERBOSE)
    def __agg_num(self):
        self.df_agg[[f"{col}_max" for col in self.num]] = self.df[['id'] + self.num].groupby('id').agg('max')
        if self.verbose: print('▮', end='')

        
    ## Has any loan
    @logme('Adding has_loans feature', verbose=VERBOSE, newline=True)
    def __add_hasloans(self):
        cols_flag = [col for col in self.df_agg.columns if col.startswith('is_zero_loans')]
        self.df_agg.loc[:, 'has_loans'] = (self.df_agg[cols_flag] == 0).any(axis=1).astype('int8')
    
    
    ## Relative OHencoded features
    @logme('Relative OH-encoded features', verbose=VERBOSE, newline=True)
    def __make_relative(self):
        coded = self.df_agg.drop(columns=['rn_max', 'credit_history_length_max', 'num_loan_months_max', 'frac_loan_months_max', 'has_loans']).columns
        df_coded = self.df_agg[coded].div(self.df_agg.rn_max, axis=0)
        self.df_agg[coded] = df_coded
        
        
    ## Adding missed columns
    def __add_missed(self):
        features = joblib.load('features.pkl')
        missed = list(set(features) - set(self.df_agg.columns))
        if missed:
            self.df_agg.loc[:, missed] = 0
            self.df_agg = self.df_agg[features]
        if self.verbose: print(f"Added {len(missed)} missed columns")

    ## Transform data
    def transform(self) -> pd.DataFrame:
        
        self.__agg_flags()
        self.__agg_cat()
        self.__new_from_enc_paym()
        self.__agg_num()
        self.__add_hasloans()
        self.__make_relative()
        
        if self.verbose: print(f"Aggregation completed. Result shape: {self.df_agg.shape}")
        
        return self.df_agg

In [15]:
gc.collect()

2215

<i>Прочитаем все parquet-файлы

In [10]:
df = pd.DataFrame()
for name in [f"data/train_data/train_data_{i}.pq" for i in range(12)]:
    print(f"Read <{name.split('/')[-1]}> ... ", end='')
    df = pd.concat((df, pd.read_parquet(full_path(name))), axis=0, ignore_index=True)
    print('done')

Read <train_data_0.pq> ... done
Read <train_data_1.pq> ... done
Read <train_data_2.pq> ... done
Read <train_data_3.pq> ... done
Read <train_data_4.pq> ... done
Read <train_data_5.pq> ... done
Read <train_data_6.pq> ... done
Read <train_data_7.pq> ... done
Read <train_data_8.pq> ... done
Read <train_data_9.pq> ... done
Read <train_data_10.pq> ... done
Read <train_data_11.pq> ... done


In [11]:
df

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,5,0,2,8,6,16,5,4,8,1,1,1,1,1,15,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,3,0,2,4,6,16,5,4,8,0,1,1,1,1,16,2,17,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26162712,2999999,8,6,5,14,13,1,15,16,2,3,0,2,0,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,3,4,1,0,0
26162713,2999999,9,5,3,2,10,15,14,17,2,3,0,2,13,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,3,4,1,0,0
26162714,2999999,10,3,16,11,13,14,8,15,5,1,0,2,8,6,16,5,4,8,1,0,1,1,1,1,2,17,0,1,1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,3,4,1,2,4,1,0,0
26162715,2999999,11,3,6,4,8,1,11,0,5,1,0,2,7,6,16,5,4,8,1,0,1,1,1,6,2,17,0,1,1,0,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,3,3,3,4,3,3,3,4,1,2,3,1,1,1


In [12]:
gc.collect()

1083

In [16]:
result = Aggregator(df, verbose=VERBOSE).transform()
result

Aggregate flags: ▮
Categorial encoding: ▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮▮
Features from enc_paym_N: ▮▮▮123456788▮
Aggregate numerical ▮
Adding has_loans feature
Relative OH-encoded features
Aggregation completed. Result shape: (3000000, 385)


,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,pre_since_opened_9,pre_since_opened_10,pre_since_opened_11,pre_since_opened_12,pre_since_opened_13,pre_since_opened_14,pre_since_opened_15,pre_since_opened_16,pre_since_opened_17,pre_since_opened_18,pre_since_opened_19,pre_since_confirmed_0,pre_since_confirmed_1,pre_since_confirmed_2,pre_since_confirmed_3,pre_since_confirmed_4,pre_since_confirmed_5,pre_since_confirmed_6,pre_since_confirmed_7,pre_since_confirmed_8,pre_since_confirmed_9,pre_since_confirmed_10,pre_since_confirmed_11,pre_since_confirmed_12,pre_since_confirmed_13,pre_since_confirmed_14,pre_since_confirmed_15,pre_since_confirmed_16,pre_since_confirmed_17,pre_pterm_0,pre_pterm_1,pre_pterm_2,pre_pterm_3,pre_pterm_4,pre_pterm_5,pre_pterm_6,pre_pterm_7,pre_pterm_8,pre_pterm_9,pre_pterm_10,pre_pterm_11,pre_pterm_12,pre_pterm_13,pre_pterm_14,pre_pterm_15,pre_pterm_16,pre_pterm_17,pre_fterm_0,pre_fterm_1,pre_fterm_2,pre_fterm_3,pre_fterm_4,pre_fterm_5,pre_fterm_6,pre_fterm_7,pre_fterm_8,pre_fterm_9,pre_fterm_10,pre_fterm_11,pre_fterm_12,pre_fterm_13,pre_fterm_14,pre_fterm_15,pre_fterm_16,pre_till_pclose_0,pre_till_pclose_1,pre_till_pclose_2,pre_till_pclose_3,pre_till_pclose_4,pre_till_pclose_5,pre_till_pclose_6,pre_till_pclose_7,pre_till_pclose_8,pre_till_pclose_9,pre_till_pclose_10,pre_till_pclose_11,pre_till_pclose_12,pre_till_pclose_13,pre_till_pclose_14,pre_till_pclose_15,pre_till_pclose_16,pre_till_fclose_0,pre_till_fclose_1,pre_till_fclose_2,pre_till_fclose_3,pre_till_fclose_4,pre_till_fclose_5,pre_till_fclose_6,pre_till_fclose_7,pre_till_fclose_8,pre_till_fclose_9,pre_till_fclose_10,pre_till_fclose_11,pre_till_fclose_12,pre_till_fclose_13,pre_till_fclose_14,pre_till_fclose_15,pre_loans_credit_limit_0,pre_loans_credit_limit_1,pre_loans_credit_limit_2,pre_loans_credit_limit_3,pre_loans_credit_limit_4,pre_loans_credit_limit_5,pre_loans_credit_limit_6,pre_loans_credit_limit_7,pre_loans_credit_limit_8,pre_loans_credit_limit_9,pre_loans_credit_limit_10,pre_loans_credit_limit_11,pre_loans_credit_limit_12,pre_loans_credit_limit_13,pre_loans_credit_limit_14,pre_loans_credit_limit_15,pre_loans_credit_limit_16,pre_loans_credit_limit_17,pre_loans_credit_limit_18,pre_loans_credit_limit_19,pre_loans_next_pay_summ_0,pre_loans_next_pay_summ_1,pre_loans_next_pay_summ_2,pre_loans_next_pay_summ_3,pre_loans_next_pay_summ_4,pre_loans_next_pay_summ_5,pre_loans_next_pay_summ_6,pre_loans_outstanding_1,pre_loans_outstanding_2,pre_loans_outstanding_3,pre_loans_outstanding_4,pre_loans_outstanding_5,pre_loans_total_overdue_0,pre_loans_total_overdue_1,pre_loans_max_overdue_sum_0,pre_loans_max_overdue_sum_1,pre_loans_max_overdue_sum_2,pre_loans_max_overdue_sum_3,pre_loans_credit_cost_rate_0,pre_loans_credit_cost_rate_1,pre_loans_credit_cost_rate_2,pre_loans_credit_cost_rate_3,pre_loans_credit_cost_rate_4,pre_loans_credit_cost_rate_5,pre_loans_credit_cost_rate_6,pre_loans_credit_cost_rate_7,pre_loans_credit_cost_rate_8,pre_loans_credit_cost_rate_9,pre_loans_credit_cost_rate_10,pre_loans_credit_cost_rate_11,pre_loans_credit_cost_rate_12,pre_loans_credit_cost_rate_13,pre_loans5_0,pre_loans5_1,pre_loans5_2,pre_loans5_3,pre_loans5_5,pre_loans5_6,pre_loans5_7,pre_loans5_8,pre_loans5_9,pre_loans5_10,pre_loans5_11,pre_loans5_13,pre_loans5_16,pre_loans530_0,pre_loans530_1,pre_loans530_2,pre_loans530_3,pre_loans530_4,pre_loans530_5,pre_loans530_6,pre_loans530_7,pre_loans530_8,pre_loans530_9,pre_loans530_10,pre_loans530_11,pre_loans530_12,pre_loans530_13,pre_loans530_14,pre_loans530_15,pre_loans530_16,pre_loans530_17,pre_loans530_18,pre_loans530_19,pre_loans3060_0,pre_loans3060_1,pre_loans3060_2,pre_loans3060_3,pre_loans3060_4,pre_loans3060_5,pre_loans3060_6,pre_loans3060_7,pre_

In [17]:
target = pd.read_csv(full_path('data/train_target.csv'))
target

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2999995,2999995,0
2999996,2999996,0
2999997,2999997,0
2999998,2999998,0


In [18]:
result = result.join(target.flag, how='left')
result.rename(columns={'flag': 'target'}, inplace=True)
result

,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,pre_since_opened_9,pre_since_opened_10,pre_since_opened_11,pre_since_opened_12,pre_since_opened_13,pre_since_opened_14,pre_since_opened_15,pre_since_opened_16,pre_since_opened_17,pre_since_opened_18,pre_since_opened_19,pre_since_confirmed_0,pre_since_confirmed_1,pre_since_confirmed_2,pre_since_confirmed_3,pre_since_confirmed_4,pre_since_confirmed_5,pre_since_confirmed_6,pre_since_confirmed_7,pre_since_confirmed_8,pre_since_confirmed_9,pre_since_confirmed_10,pre_since_confirmed_11,pre_since_confirmed_12,pre_since_confirmed_13,pre_since_confirmed_14,pre_since_confirmed_15,pre_since_confirmed_16,pre_since_confirmed_17,pre_pterm_0,pre_pterm_1,pre_pterm_2,pre_pterm_3,pre_pterm_4,pre_pterm_5,pre_pterm_6,pre_pterm_7,pre_pterm_8,pre_pterm_9,pre_pterm_10,pre_pterm_11,pre_pterm_12,pre_pterm_13,pre_pterm_14,pre_pterm_15,pre_pterm_16,pre_pterm_17,pre_fterm_0,pre_fterm_1,pre_fterm_2,pre_fterm_3,pre_fterm_4,pre_fterm_5,pre_fterm_6,pre_fterm_7,pre_fterm_8,pre_fterm_9,pre_fterm_10,pre_fterm_11,pre_fterm_12,pre_fterm_13,pre_fterm_14,pre_fterm_15,pre_fterm_16,pre_till_pclose_0,pre_till_pclose_1,pre_till_pclose_2,pre_till_pclose_3,pre_till_pclose_4,pre_till_pclose_5,pre_till_pclose_6,pre_till_pclose_7,pre_till_pclose_8,pre_till_pclose_9,pre_till_pclose_10,pre_till_pclose_11,pre_till_pclose_12,pre_till_pclose_13,pre_till_pclose_14,pre_till_pclose_15,pre_till_pclose_16,pre_till_fclose_0,pre_till_fclose_1,pre_till_fclose_2,pre_till_fclose_3,pre_till_fclose_4,pre_till_fclose_5,pre_till_fclose_6,pre_till_fclose_7,pre_till_fclose_8,pre_till_fclose_9,pre_till_fclose_10,pre_till_fclose_11,pre_till_fclose_12,pre_till_fclose_13,pre_till_fclose_14,pre_till_fclose_15,pre_loans_credit_limit_0,pre_loans_credit_limit_1,pre_loans_credit_limit_2,pre_loans_credit_limit_3,pre_loans_credit_limit_4,pre_loans_credit_limit_5,pre_loans_credit_limit_6,pre_loans_credit_limit_7,pre_loans_credit_limit_8,pre_loans_credit_limit_9,pre_loans_credit_limit_10,pre_loans_credit_limit_11,pre_loans_credit_limit_12,pre_loans_credit_limit_13,pre_loans_credit_limit_14,pre_loans_credit_limit_15,pre_loans_credit_limit_16,pre_loans_credit_limit_17,pre_loans_credit_limit_18,pre_loans_credit_limit_19,pre_loans_next_pay_summ_0,pre_loans_next_pay_summ_1,pre_loans_next_pay_summ_2,pre_loans_next_pay_summ_3,pre_loans_next_pay_summ_4,pre_loans_next_pay_summ_5,pre_loans_next_pay_summ_6,pre_loans_outstanding_1,pre_loans_outstanding_2,pre_loans_outstanding_3,pre_loans_outstanding_4,pre_loans_outstanding_5,pre_loans_total_overdue_0,pre_loans_total_overdue_1,pre_loans_max_overdue_sum_0,pre_loans_max_overdue_sum_1,pre_loans_max_overdue_sum_2,pre_loans_max_overdue_sum_3,pre_loans_credit_cost_rate_0,pre_loans_credit_cost_rate_1,pre_loans_credit_cost_rate_2,pre_loans_credit_cost_rate_3,pre_loans_credit_cost_rate_4,pre_loans_credit_cost_rate_5,pre_loans_credit_cost_rate_6,pre_loans_credit_cost_rate_7,pre_loans_credit_cost_rate_8,pre_loans_credit_cost_rate_9,pre_loans_credit_cost_rate_10,pre_loans_credit_cost_rate_11,pre_loans_credit_cost_rate_12,pre_loans_credit_cost_rate_13,pre_loans5_0,pre_loans5_1,pre_loans5_2,pre_loans5_3,pre_loans5_5,pre_loans5_6,pre_loans5_7,pre_loans5_8,pre_loans5_9,pre_loans5_10,pre_loans5_11,pre_loans5_13,pre_loans5_16,pre_loans530_0,pre_loans530_1,pre_loans530_2,pre_loans530_3,pre_loans530_4,pre_loans530_5,pre_loans530_6,pre_loans530_7,pre_loans530_8,pre_loans530_9,pre_loans530_10,pre_loans530_11,pre_loans530_12,pre_loans530_13,pre_loans530_14,pre_loans530_15,pre_loans530_16,pre_loans530_17,pre_loans530_18,pre_loans530_19,pre_loans3060_0,pre_loans3060_1,pre_loans3060_2,pre_loans3060_3,pre_loans3060_4,pre_loans3060_5,pre_loans3060_6,pre_loans3060_7,pre_

In [19]:
result.target.value_counts()

target
0    2893558
1     106442
Name: count, dtype: int64

In [20]:
gc.collect()

67

In [21]:
result.to_parquet(full_path('data/res_11.pq'))